In [22]:
import plotly
plotly.__version__
import pandas as pd

In [23]:
mdata = pd.read_csv("DataSets/belly_button_metadata.csv")
mdata.head()

,sample,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [24]:
bbdata = pd.read_csv("Datasets/belly_button_data.csv")
bbdata.head()

,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, desc

In [48]:
# Create engine using the database file
engine = create_engine("sqlite:///StarterCode/Belly_Button_Biodiversity/db/belly_button_biodiversity.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [49]:
# Print all of the classes mapped to the Base# Print 
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [50]:
Otu = Base.classes.otu
Samples = Base.classes.samples
Samples_Metadata = Base.classes.samples_metadata

In [51]:
session = Session(engine)

In [54]:
def getOtuDistribution(sample_number):
    fields = [sample_number]
    results = session.query(Samples).options(load_only(sample_number)).order_by(desc(sample_number))
    
    distribution = []
    labels = []
    values_raw = []
    
    for r in results[:10]:
        row = r.__dict__    
        labels.append(f"Otu - {row['otu_id']}")
        values_raw.append(row[f"{sample_number}"])
    
    values = [round((x/sum(values_raw))*100,2) for x in values_raw]
        
    return {
        'labels':labels,
        'values':values
    }

In [56]:
getOtuDistribution('BB_966')

{'labels': ['Otu - 943',
  'Otu - 922',
  'Otu - 1453',
  'Otu - 1795',
  'Otu - 1929',
  'Otu - 944',
  'Otu - 928',
  'Otu - 1895',
  'Otu - 3476',
  'Otu - 909'],
 'values': [41.88, 26.72, 6.72, 6.41, 4.69, 3.12, 2.97, 2.81, 2.66, 2.03]}

In [57]:
def getPersonInfo(sample_number):
    sample_id = sample_number[3:]
    results = session.query(Samples_Metadata).\
    filter(Samples_Metadata.SAMPLEID == sample_id).first()
    
    print(results)
    
    return {
        'age':results.AGE,
        'gender':results.GENDER,
        'ethnicity':results.ETHNICITY,
        'location':results.LOCATION,
        'source':results.EVENT
    }

In [58]:
getPersonInfo('BB_948')

{'age': 20,
 'gender': 'M',
 'ethnicity': 'Caucasian',
 'location': 'ChapelHill/NC',
 'source': 'BellyButtonsScienceOnline'}

In [59]:
def getOtuSampleRelation():
    
    results = session.query(Samples).all()
    x = []
    y = []
    
    for r in results:
        row = r.__dict__
        sample_values = list(row.values())
        x.append(r.otu_id)
        y.append(sum(sample_values[1:]))

    return {
        'x':x,
        'y':y
    }

In [60]:
getOtuSampleRelation()

{'x': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  15

In [61]:
def SampleList():
    columns_list = Samples.__table__.columns.keys()
    return columns_list[1:]

In [62]:
SampleList()

['BB_940',
 'BB_941',
 'BB_943',
 'BB_944',
 'BB_945',
 'BB_946',
 'BB_947',
 'BB_948',
 'BB_949',
 'BB_950',
 'BB_952',
 'BB_953',
 'BB_954',
 'BB_955',
 'BB_956',
 'BB_958',
 'BB_959',
 'BB_960',
 'BB_961',
 'BB_962',
 'BB_963',
 'BB_964',
 'BB_966',
 'BB_967',
 'BB_968',
 'BB_969',
 'BB_970',
 'BB_971',
 'BB_972',
 'BB_973',
 'BB_974',
 'BB_975',
 'BB_978',
 'BB_1233',
 'BB_1234',
 'BB_1235',
 'BB_1236',
 'BB_1237',
 'BB_1238',
 'BB_1242',
 'BB_1243',
 'BB_1246',
 'BB_1253',
 'BB_1254',
 'BB_1258',
 'BB_1259',
 'BB_1260',
 'BB_1264',
 'BB_1265',
 'BB_1273',
 'BB_1275',
 'BB_1276',
 'BB_1277',
 'BB_1278',
 'BB_1279',
 'BB_1280',
 'BB_1281',
 'BB_1282',
 'BB_1283',
 'BB_1284',
 'BB_1285',
 'BB_1286',
 'BB_1287',
 'BB_1288',
 'BB_1289',
 'BB_1290',
 'BB_1291',
 'BB_1292',
 'BB_1293',
 'BB_1294',
 'BB_1295',
 'BB_1296',
 'BB_1297',
 'BB_1298',
 'BB_1308',
 'BB_1309',
 'BB_1310',
 'BB_1374',
 'BB_1415',
 'BB_1439',
 'BB_1441',
 'BB_1443',
 'BB_1486',
 'BB_1487',
 'BB_1489',
 'BB_1490',
 

In [64]:
results = session.query(Samples).options(load_only('BB_958')).order_by('otu_id')
x = []
y = []

for r in results:
    row = r.__dict__
    sample_values = list(row.values())
    print(f"{r.otu_id}-{sample_values[1]}")
    x.append(r.otu_id)
    y.append(sample_values[1])

print({
    'x':x,
    'y':y
})

1-1
2-2
3-3
4-4
5-5
6-6
7-7
8-8
9-9
10-10
11-11
12-12
13-13
14-14
15-15
16-16
17-17
18-18
19-19
20-20
21-21
22-22
23-23
24-24
25-25
26-26
27-27
28-28
29-29
30-30
31-31
32-32
33-33
34-34
35-35
36-36
37-37
38-38
39-39
40-40
41-41
42-42
43-43
44-44
45-45
46-46
47-47
48-48
49-49
50-50
51-51
52-52
53-53
54-54
55-55
56-56
57-57
58-58
59-59
60-60
61-61
62-62
63-63
64-64
65-65
66-66
67-67
68-68
69-69
70-70
71-71
72-72
73-73
74-74
75-75
76-76
77-77
78-78
79-79
80-80
81-81
82-82
83-83
84-84
85-85
86-86
87-87
88-88
89-89
90-90
91-91
92-92
93-93
94-94
95-95
96-96
97-97
98-98
99-99
100-100
101-101
102-102
103-103
104-104
105-105
106-106
107-107
108-108
109-109
110-110
111-111
112-112
113-113
114-114
115-115
116-116
117-117
118-118
119-119
120-120
121-121
122-122
123-123
124-124
125-125
126-126
127-127
128-128
129-129
130-130
131-131
132-132
133-133
134-134
135-135
136-136
137-137
138-138
139-139
140-140
141-141
142-142
143-143
144-144
145-145
146-146
147-147
148-148
149-149
150-150
151-151
152-152


2534-2534
2535-2535
2536-2536
2537-2537
2538-2538
2539-2539
2540-2540
2541-2541
2542-2542
2543-2543
2544-2544
2545-2545
2546-2546
2547-2547
2548-2548
2549-2549
2550-2550
2551-2551
2552-2552
2553-2553
2554-2554
2555-2555
2556-2556
2557-2557
2558-2558
2559-2559
2560-2560
2561-2561
2562-2562
2563-2563
2564-2564
2565-2565
2566-2566
2567-2567
2568-2568
2569-2569
2570-2570
2571-2571
2572-2572
2573-2573
2574-2574
2575-2575
2576-2576
2577-2577
2578-2578
2579-2579
2580-2580
2581-2581
2582-2582
2583-2583
2584-2584
2585-2585
2586-2586
2587-2587
2588-2588
2589-2589
2590-2590
2591-2591
2592-2592
2593-2593
2594-2594
2595-2595
2596-2596
2597-2597
2598-2598
2599-2599
2600-2600
2601-2601
2602-2602
2603-2603
2604-2604
2605-2605
2606-2606
2607-2607
2608-2608
2609-2609
2610-2610
2611-2611
2612-2612
2613-2613
2614-2614
2615-2615
2616-2616
2617-2617
2618-2618
2619-2619
2620-2620
2621-2621
2622-2622
2623-2623
2624-2624
2625-2625
2626-2626
2627-2627
2628-2628
2629-2629
2630-2630
2631-2631
2632-2632
2633-2633


3623-3623
3624-3624
3625-3625
3626-3626
3627-3627
3628-3628
3629-3629
3630-3630
3631-3631
3632-3632
3633-3633
3634-3634
3635-3635
3636-3636
3637-3637
3638-3638
3639-3639
3640-3640
3641-3641
3642-3642
3643-3643
3644-3644
3645-3645
3646-3646
3647-3647
3648-3648
3649-3649
3650-3650
3651-3651
3652-3652
3653-3653
3654-3654
3655-3655
3656-3656
3657-3657
3658-3658
3659-3659
3660-3660
3661-3661
3662-3662
3663-3663
3664-3664
3665-3665
3666-3666
3667-3667
3668-3668
3669-3669
3670-3670
3671-3671
3672-3672
3673-3673
3674-3674
{'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 1